In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load datasets
train_data_path = 'top 5+ stock 2019-2023.xlsx'
forecast_data_path = 'top 5 2024_2033.xlsx'

# Read the data
train_data = pd.ExcelFile(train_data_path).parse(0)  # Assuming the first sheet contains data
forecast_data = pd.ExcelFile(forecast_data_path).parse(0)  # Assuming the first sheet contains data

# Inspect the data
print("Training data head:")
print(train_data.head())

print("\nForecast data head:")
print(forecast_data.head())

# Filter and reshape the training data
train_data = train_data[train_data['Parameters'].str.strip().str.lower() == 'repurchase of common stock']
if train_data.empty:
    raise ValueError("No matching data found in training data for 'Repurchase of Common Stock'. Check the dataset.")

train_data = train_data.melt(id_vars=['Company_name', 'Parameters'], 
                             var_name='Year', 
                             value_name='Repurchase')
train_data['Year'] = train_data['Year'].astype(int)

# Select features and target
X = train_data[['Company_name', 'Year']]  # Features
y = train_data['Repurchase']  # Target variable

# Encode categorical variables (if any)
X = pd.get_dummies(X, columns=['Company_name'], drop_first=True)

# Check if X is empty after encoding
if X.empty:
    raise ValueError("Encoded feature set is empty. Check categorical encoding.")

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Build the neural network model
model = Sequential([
    Dense(64, activation='relu', input_dim=X_train.shape[1]),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=16, verbose=1)

# Preprocess the forecast data
forecast_data = forecast_data[forecast_data['Parameters'].str.strip().str.lower() == 'repurchase of common stock']
if forecast_data.empty:
    raise ValueError("No matching data found in forecast data for 'Repurchase of Common Stock'. Check the dataset.")

forecast_data = forecast_data.melt(id_vars=['Company_name', 'Parameters'], 
                                   var_name='Year', 
                                   value_name='Repurchase')
forecast_data['Year'] = forecast_data['Year'].astype(int)

forecast_X = forecast_data[['Company_name', 'Year']]
forecast_X = pd.get_dummies(forecast_X, columns=['Company_name'], drop_first=True)
forecast_X_scaled = scaler.transform(forecast_X)

# Predict repurchase values for 2024-2033
forecast_data['Repurchase_Predicted'] = model.predict(forecast_X_scaled)

# Reshape the output to match the required format
output_data = forecast_data[['Company_name', 'Year']].copy()
output_data['Parameter'] = 'Repurchase of Common Stock'
output_data['Value'] = forecast_data['Repurchase_Predicted']

# Save the results
output_path = 'repurchase_predictions_2024_2033.xlsx'
output_data.to_excel(output_path, index=False)
print(f"Predictions saved to {output_path}")


Training data head:
  Company_name                                   Parameters    2019    2020  \
0          BKR                   repurchase of common stock     0.0     0.0   
1          BKR                               dividends paid  -630.0  -278.0   
2          BKR  net cash flows used in financing activities  -695.0 -1007.0   
3          BKR  net cash flows used in investing activities -1445.0  -486.0   
4          BKR     proceeds from sales of equity securities  2669.0  2666.0   

     2021    2022    2023  
0  -434.0  -828.0  -538.0  
1  -161.0  -435.0  -576.0  
2  -838.0 -1798.0 -1671.0  
3  -534.0  -967.0 -1659.0  
4  2665.0  2664.0  2663.0  

Forecast data head:
  Company_name                                   Parameters      2024  \
0          BKR                               dividends paid  -570.824   
1          BKR  net cash flows used in financing activities -1917.007   
2          BKR  net cash flows used in investing activities -1714.013   
3          BKR     proce

C:\Users\HP\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 518ms/step - loss: 1604371.3750 - mae: 535.7933 - val_loss: 5266856.0000 - val_mae: 1330.5236
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step - loss: 1659614.6250 - mae: 579.1215 - val_loss: 5266442.0000 - val_mae: 1330.4481
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - loss: 1610938.6250 - mae: 549.4907 - val_loss: 5266068.0000 - val_mae: 1330.3800
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step - loss: 1632803.7500 - mae: 573.8770 - val_loss: 5265672.0000 - val_mae: 1330.3092
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - loss: 1649783.3750 - mae: 567.7875 - val_loss: 5265272.5000 - val_mae: 1330.2366
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step - loss: 1616679.7500 - mae: 550.2119 - val_loss: 5264880.5000 - val_mae: 1330.1641
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step - loss: 994341.6875 - mae: 413.1843 - val_loss: 5264519.0000 - val_mae: 1330.0948
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step - loss: 1609492.0000

ValueError: No matching data found in forecast data for 'Repurchase of Common Stock'. Check the dataset.

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load datasets
train_data_path = 'top 5+ stock 2019-2023.xlsx'
forecast_data_path = 'top 5 2024_2033.xlsx'

# Read the data
train_data = pd.ExcelFile(train_data_path).parse(0)  # Assuming the first sheet contains data
forecast_data = pd.ExcelFile(forecast_data_path).parse(0)  # Assuming the first sheet contains data

# Inspect the data
print("Training data head:")
print(train_data.head())

print("\nForecast data head:")
print(forecast_data.head())

# Check unique values in the Parameters column
print("\nUnique values in Parameters column of training data:")
print(train_data['Parameters'].unique())

print("\nUnique values in Parameters column of forecast data:")
print(forecast_data['Parameters'].unique())

# Filter and reshape the training data
train_data = train_data[train_data['Parameters'].str.strip().str.lower() == 'repurchase of common stock']
if train_data.empty:
    raise ValueError("No matching data found in training data for 'Repurchase of Common Stock'. Check the dataset.")

train_data = train_data.melt(id_vars=['Company_name', 'Parameters'], 
                             var_name='Year', 
                             value_name='Repurchase')
train_data['Year'] = train_data['Year'].astype(int)  # Ensure year is an integer

# Select features and target
X = train_data[['Company_name', 'Year']]  # Features (Company name and Year for simplicity in this example)
y = train_data['Repurchase']  # Target variable

# Encode categorical variables (if any)
X = pd.get_dummies(X, columns=['Company_name'], drop_first=True)

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Build the neural network model
model = Sequential([
    Dense(64, activation='relu', input_dim=X_train.shape[1]),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=16, verbose=1)

# Preprocess the forecast data
forecast_data = forecast_data[forecast_data['Parameters'].str.strip().str.lower() == 'repurchase of common stock']
if forecast_data.empty:
    raise ValueError("No matching data found in forecast data for 'Repurchase of Common Stock'. Check the dataset.")

forecast_data = forecast_data.melt(id_vars=['Company_name', 'Parameters'], 
                                   var_name='Year', 
                                   value_name='Repurchase')
forecast_data['Year'] = forecast_data['Year'].astype(int)

forecast_X = forecast_data[['Company_name', 'Year']]
forecast_X = pd.get_dummies(forecast_X, columns=['Company_name'], drop_first=True)

# Align columns in forecast data with training data
forecast_X = forecast_X.reindex(columns=X.columns, fill_value=0)

# Scale the features
forecast_X_scaled = scaler.transform(forecast_X)

# Predict repurchase values for 2024-2033
forecast_data['Repurchase_Predicted'] = model.predict(forecast_X_scaled)

# Reshape the output to match the required format
output_data = forecast_data[['Company_name', 'Year']].copy()
output_data['Parameter'] = 'Repurchase of Common Stock'
output_data['Value'] = forecast_data['Repurchase_Predicted']

# Save the results
output_path = 'repurchase_predictions_2024_2033.xlsx'
output_data.to_excel(output_path, index=False)
print(f"Predictions saved to {output_path}")


Training data head:
  Company_name                                   Parameters    2019    2020  \
0          BKR                   repurchase of common stock     0.0     0.0   
1          BKR                               dividends paid  -630.0  -278.0   
2          BKR  net cash flows used in financing activities  -695.0 -1007.0   
3          BKR  net cash flows used in investing activities -1445.0  -486.0   
4          BKR     proceeds from sales of equity securities  2669.0  2666.0   

     2021    2022    2023  
0  -434.0  -828.0  -538.0  
1  -161.0  -435.0  -576.0  
2  -838.0 -1798.0 -1671.0  
3  -534.0  -967.0 -1659.0  
4  2665.0  2664.0  2663.0  

Forecast data head:
  Company_name                                   Parameters      2024  \
0          BKR                               dividends paid  -570.824   
1          BKR  net cash flows used in financing activities -1917.007   
2          BKR  net cash flows used in investing activities -1714.013   
3          BKR     proce

C:\Users\HP\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 429ms/step - loss: 1594497.1250 - mae: 532.7328 - val_loss: 5266381.5000 - val_mae: 1330.6180
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step - loss: 1666685.7500 - mae: 585.1063 - val_loss: 5265975.5000 - val_mae: 1330.5443
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - loss: 1088871.0000 - mae: 477.0102 - val_loss: 5265594.5000 - val_mae: 1330.4740
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step - loss: 1647408.6250 - mae: 564.2136 - val_loss: 5265201.0000 - val_mae: 1330.4012
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - loss: 1622556.1250 - mae: 565.4702 - val_loss: 5264820.5000 - val_mae: 1330.3303
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - loss: 1671172.8750 - mae: 593.3452 - val_loss: 5264440.5000 - val_mae: 1330.2593
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step - loss: 1095915.7500 - mae: 483.9811 - val_loss: 5264071.5000 - val_mae: 1330.1896
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step - loss: 1670879.6250 - mae: 59

ValueError: No matching data found in forecast data for 'Repurchase of Common Stock'. Check the dataset.

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load datasets
train_data_path = 'top 5+ stock 2019-2023.xlsx'
forecast_data_path = 'top 5 2024_2033.xlsx'

# Read the data
train_data = pd.ExcelFile(train_data_path).parse(0)  # Assuming the first sheet contains data
forecast_data = pd.ExcelFile(forecast_data_path).parse(0)  # Assuming the first sheet contains data

# Inspect the data
print("Training data head:")
print(train_data.head())

print("\nForecast data head:")
print(forecast_data.head())

# Normalize and filter training data
train_data['Parameters'] = train_data['Parameters'].str.strip().str.lower()
train_data = train_data[train_data['Parameters'] == 'repurchase of common stock']

if train_data.empty:
    raise ValueError("No matching data found in training data for 'Repurchase of Common Stock'. Check the dataset formatting.")

# Reshape training data
train_data = train_data.melt(id_vars=['Company_name', 'Parameters'], 
                             var_name='Year', 
                             value_name='Repurchase')
train_data['Year'] = train_data['Year'].astype(int)

# Select features and target
X = train_data[['Company_name', 'Year']]
y = train_data['Repurchase']

# Encode categorical variables
X = pd.get_dummies(X, columns=['Company_name'], drop_first=True)

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Build the neural network model
model = Sequential([
    Dense(64, activation='relu', input_dim=X_train.shape[1]),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=16, verbose=1)

# Normalize and filter forecast data
forecast_data['Parameters'] = forecast_data['Parameters'].str.strip().str.lower()
forecast_data = forecast_data[forecast_data['Parameters'] == 'repurchase of common stock']

if forecast_data.empty:
    raise ValueError(f"No matching data found in forecast data for 'Repurchase of Common Stock'. "
                     f"Available parameters: {forecast_data['Parameters'].unique()}")

# Reshape forecast data
forecast_data = forecast_data.melt(id_vars=['Company_name', 'Parameters'], 
                                   var_name='Year', 
                                   value_name='Repurchase')
forecast_data['Year'] = forecast_data['Year'].astype(int)

# Prepare features for prediction
forecast_X = forecast_data[['Company_name', 'Year']]
forecast_X = pd.get_dummies(forecast_X, columns=['Company_name'], drop_first=True)

# Align columns with training data
missing_cols = set(X.columns) - set(forecast_X.columns)
for col in missing_cols:
    forecast_X[col] = 0  # Add missing columns as zeros

forecast_X = forecast_X[X.columns]  # Ensure same column order
forecast_X_scaled = scaler.transform(forecast_X)

# Predict repurchase values for 2024-2033
forecast_data['Repurchase_Predicted'] = model.predict(forecast_X_scaled)

# Reshape the output to match the required format
output_data = forecast_data[['Company_name', 'Year']].copy()
output_data['Parameter'] = 'Repurchase of Common Stock'
output_data['Value'] = forecast_data['Repurchase_Predicted']

# Save the results
output_path = 'repurchase_predictions_2024_2033.xlsx'
output_data.to_excel(output_path, index=False)
print(f"Predictions saved to {output_path}")


Training data head:
  Company_name                                   Parameters    2019    2020  \
0          BKR                   repurchase of common stock     0.0     0.0   
1          BKR                               dividends paid  -630.0  -278.0   
2          BKR  net cash flows used in financing activities  -695.0 -1007.0   
3          BKR  net cash flows used in investing activities -1445.0  -486.0   
4          BKR     proceeds from sales of equity securities  2669.0  2666.0   

     2021    2022    2023  
0  -434.0  -828.0  -538.0  
1  -161.0  -435.0  -576.0  
2  -838.0 -1798.0 -1671.0  
3  -534.0  -967.0 -1659.0  
4  2665.0  2664.0  2663.0  

Forecast data head:
  Company_name                                   Parameters      2024  \
0          BKR                               dividends paid  -570.824   
1          BKR  net cash flows used in financing activities -1917.007   
2          BKR  net cash flows used in investing activities -1714.013   
3          BKR     proce

C:\Users\HP\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 386ms/step - loss: 1580342.6250 - mae: 534.7938 - val_loss: 5268265.0000 - val_mae: 1330.7869
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - loss: 1576741.0000 - mae: 529.0696 - val_loss: 5268058.0000 - val_mae: 1330.7369
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step - loss: 1613204.7500 - mae: 548.3751 - val_loss: 5267843.0000 - val_mae: 1330.6848
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - loss: 1677623.1250 - mae: 607.8572 - val_loss: 5267628.5000 - val_mae: 1330.6327
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - loss: 1613751.0000 - mae: 550.0004 - val_loss: 5267411.5000 - val_mae: 1330.5807
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step - loss: 1051765.0000 - mae: 459.0459 - val_loss: 5267194.0000 - val_mae: 1330.5289
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - loss: 1101843.5000 - mae: 493.1566 - val_loss: 5266962.5000 - val_mae: 1330.4750
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step - loss: 1676318.8750 - mae: 60

ValueError: No matching data found in forecast data for 'Repurchase of Common Stock'. Available parameters: []

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Load the training data
training_data = pd.read_excel("top 5+ stock 2019-2023.xlsx")

# Normalize parameter names in training data
training_data["Parameters"] = training_data["Parameters"].str.lower().str.strip()

# Transform the data for training
training_data_melted = training_data.melt(
    id_vars=["Company_name", "Parameters"], 
    var_name="Year", 
    value_name="Value"
)

# Pivot the data to have years as columns
training_data_pivot = training_data_melted.pivot_table(
    index=["Company_name", "Parameters"],
    columns="Year",
    values="Value"
).reset_index()

# Extract features and target columns
features = training_data_pivot.iloc[:, 2:-1].values  # All years except the last one
target = training_data_pivot.iloc[:, -1].values  # The last year as the target

# Normalize the features and target
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)
target_scaled = scaler.fit_transform(target.reshape(-1, 1))

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    features_scaled, target_scaled, test_size=0.2, random_state=42
)

# Define the neural network model
model = Sequential([
    Dense(128, activation="relu", input_dim=X_train.shape[1]),
    Dropout(0.2),
    Dense(64, activation="relu"),
    Dropout(0.2),
    Dense(1, activation="linear")
])

model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=32
)

# Load the forecast data
forecast_data = pd.read_excel("top 5 2024_2033.xlsx")

# Normalize parameter names in forecast data
forecast_data["Parameters"] = forecast_data["Parameters"].str.lower().str.strip()

# Define the parameter to forecast
parameter_to_forecast = "repurchase of common stock".lower().strip()

# Check if the parameter exists in the forecast data
if parameter_to_forecast not in forecast_data["Parameters"].values:
    available_parameters = forecast_data["Parameters"].unique()
    raise ValueError(
        f"No matching data found in forecast data for '{parameter_to_forecast}'.\n"
        f"Available parameters: {available_parameters}"
    )

# Filter the forecast data for the desired parameter
forecast_data_filtered = forecast_data[forecast_data["Parameters"] == parameter_to_forecast]

# Prepare the forecast features
forecast_features = forecast_data_filtered.iloc[:, 2:].values
forecast_features_scaled = scaler.transform(forecast_features)

# Predict future values
predicted_values_scaled = model.predict(forecast_features_scaled)

# Inverse scale the predictions
predicted_values = scaler.inverse_transform(predicted_values_scaled)

# Add the predictions to the forecast data
forecast_data_filtered.loc[:, "Predicted Values"] = predicted_values

# Save the updated forecast data
forecast_data_filtered.to_csv("forecasted_data_with_predictions.csv", index=False)

print("Forecasting complete. Results saved to 'forecasted_data_with_predictions.csv'.")


Epoch 1/50


C:\Users\HP\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 1.2923 - mae: 1.1199 - val_loss: 1.2412 - val_mae: 1.1133
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step - loss: 1.1540 - mae: 1.0525 - val_loss: 1.1356 - val_mae: 1.0649
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step - loss: 1.0951 - mae: 1.0319 - val_loss: 1.0398 - val_mae: 1.0191
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step - loss: 0.9537 - mae: 0.9602 - val_loss: 0.9496 - val_mae: 0.9740
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step - loss: 0.8295 - mae: 0.8899 - val_loss: 0.8652 - val_mae: 0.9297
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step - loss: 0.7931 - mae: 0.8724 - val_loss: 0.7864 - val_mae: 0.8864
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 422ms/step - loss: 0.7432 - mae: 0.8505 - val_loss: 0.7139 - val_mae: 0.8446
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step - loss: 0.6859 - mae: 0.8116 - val_loss: 0.6450 - val_mae: 0.8028
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step - loss: 0.5754 - mae: 0.7441 - val_l

ValueError: No matching data found in forecast data for 'repurchase of common stock'.
Available parameters: ['dividends paid' 'net cash flows used in financing activities'
 'net cash flows used in investing activities'
 'proceeds from sales of equity securities' 'total current assets'
 'all other current assets' 'other intangible assets, net'
 'other non-operating income' 'proceeds from issuance of long-term debt'
 'property, plant, and equipment less accumulated depreciation'
 'accumulated other comprehensive loss' 'contract & other deferred assets'
 'depreciation & amortization' 'net cash paid for acquisitions'
 'changes in operating assets & liabilities (total)'
 'cost of services sold' 'gross profit' 'sales of services'
 'ebitda margin' 'increase (decrease) in cash & cash equivalents'
 'operating cash flow margin' 'operating income' 'sales of goods'
 'all other current liabilities']